# About: MoodleのURLを変更する

---

MoodleのURLを変更します。

## パラメータ設定

### グループ名の指定

このNotebookの操作対象となるAnsibleのグループ名を設定します。

既存のグループ名を確認するために`group_vars`にあるディレクトリの一覧を表示します。

In [ ]:
!ls -1 group_vars/

上のセルの出力結果を参考にしてAnsibleのグループ名を次のセルに指定してください。

In [ ]:
# (例)
# target_group = 'Moodle'

target_group = 

#### チェック

指定された `target_group` の値が適切なものかチェックします。

`target_group` に対応する設定ファイルが存在していることを確認します。

In [ ]:
from pathlib import Path

target_dir = Path('group_vars') / target_group
if not (target_dir.is_dir() and len(list(target_dir.glob("*"))) > 0):
    target_file = Path('group_vars') / f"{target_group}.yml"
    if target_file.is_file():
        target_dir.mkdir(exist_ok=True)
        target_file.rename(target_dir / "00-moodle.yml")
    else:
        raise RuntimeError(f"ERROR: not exists {target_group}")

`target_group`で指定したホストにAnsibleで到達可能であることを確認します。

In [ ]:
!ansible {target_group} -m ping

### MoodleのURL

次のセルに新しいURLを指定してください。

In [ ]:
# (例)
# moodle_url = 'http://moodle.example.org'
# moodle_url = 'http://172.30.2.100'
# moodle_url = 'https://moodle.example.org'

moodle_url = 

#### パラメータの保存

この節で指定したパラメータの値をファイルに保存します。

値の保存を行う前に、入力されたパラメータに対して簡易なチェックを行います。エラーになった場合はその後に表示される指示に従ってください。

In [ ]:
%run scripts/utils.py
check_parameters('moodle_url', nb_vars=locals())

パラメータの値を group_vars ファイルに保存します。

In [ ]:
from urllib.parse import urlparse
%run scripts/group.py

res = urlparse(moodle_url)
update_group_vars(
    target_group,
    moodle_url=moodle_url,
    server_name=res.netloc,
)

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{target_group}/20-moodle.yml

## URLの変更

Moodleコンテナを新しく指定されたURLで起動し直します。

### メンテナンスモードへの切り替え

Moodleをメンテナンスモードに切り替えます。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle docker compose exec -T moodle \
    /usr/bin/php /var/www/html/admin/cli/maintenance.php --enable'

### docker-compose.yml

新たな URL の値で `docker-compose.yml`を生成し直します。

In [ ]:
!ansible {target_group} -Dv -m template \
    -a 'src=template/docker-compose.yml.j2 dest=/srv/moodle/docker-compose.yml backup=yes'

### コンテナの起動

コンテナを起動し直します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker compose up -d'

コンテナの状態を確認します。`State`が`Up`となっていることを確認してください。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker compose ps'

### メンテナンスモードの解除

メンテナンスモードを解除します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle docker compose exec -T moodle \
    /usr/bin/php /var/www/html/admin/cli/maintenance.php --disable'

## Moodle を利用できることを確認

変更したURLにアクセスしMoodle が利用できることを確認します。

次のセルを実行すると変更したURLのリンクが表示されます。リンクをクリックしてMoodle環境にログインできることを確認してください。

In [ ]:
from IPython.core.display import HTML
HTML(u'<a href="{0}/report/loglive/index.php" target="_blank">{0}/report/loglive/index.php</a>'.format(moodle_url))